In [ ]:
## Import necessary packages

%matplotlib qt
# On Macs use osx
# For Windows use qt

import numpy as np
from numpy.random import rand
from landscape import Landscape # Import methods from inside file landscape.py

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm

In [ ]:
### Define simulation grid and initial conditions

# small random features in topography to begin erosion
def initial_conditions(NX,NY):
    Z = rand(NX,NY)
    return Z

NX = 70 #number of rows
NY = 70 #number of columns

d  = 5 # grid spacing in meters
dx = d # keep dx=dy for simplicity
dy = d

LX=NX*dx
LY=NY*dy

Z = initial_conditions(NX,NY)

x = np.arange(NX)
y = np.arange(NY)
X,Y = np.meshgrid(y,x) #strange that y goes first !!!

In [ ]:
### Physical Parameters
K = 1.0e-6 # meters^(1-2m)/yr

D = 0.005 # m^2/yr

# uplift rate
uplift = 0.03 / 600.

In [ ]:
### Model parameters

# Set the time step dt
# Remember for 2D diffusion problems we need eta = D * dt / dx^2 < 1/4
# but because of the fluvial erosion terms please use eta < 1/8 or less
dt = ...      
print(' dt[years] = ',dt)

#Area exponent A^m, default m=1
m=1

#gradient exponent g^n, default n=1
n=1

#erosion threshold 
theta_c = 10 

In [ ]:
# Total simulation time
T = 2000.0 * 625.0

# total number of iterations
n_iter = int(np.round(T/dt))
print('Number of interations: ',n_iter)

In [ ]:
# Initialize landscape 
ls = Landscape(NX,NY)
ls.pool_check(Z,NX,NY)
ls.A = np.zeros((NX,NY))

In [ ]:
# Set-up figure
def init_figure():
    fig = plt.figure(figsize=(12.,6.))
    plt.show()
    return fig

def update_figure():
        plt.clf()
        ax1 = fig.add_subplot(121,projection='3d')

        # use equal x-y aspect with an explicit vertical exageration
        vert_exag = 4.
        ax1.set_xlim3d(0,max(NX,NY))
        ax1.set_ylim3d(0,max(NX,NY))
        ax1.set_zlim3d(0,max(NX,NY) / vert_exag)

        ax1.set_title('Surface Relief x '+str(vert_exag))

#        surf = ax1.plot_surface(X,Y,Z, color='yellowgreen', rstride=1, cstride=1,
#                antialiased=False,linewidth=0)
        surf = ax1.plot_surface(X,Y,Z, cmap = cm.terrain, rstride=1, cstride=1,
                antialiased=False,linewidth=0)

        ax2 = fig.add_subplot(122,aspect='equal')
        ax2.set_title('Elevation')

        #im = ax2.pcolor(Z,cmap=cm.terrain)
        im = ax2.pcolor(Z,cmap=cm.coolwarm)
        cs = ax2.contour(Z,6,colors='k')

        # Add a color bar which maps values to colors.
        cbar = fig.colorbar(im, shrink=0.5, aspect=5)
        # Add the contour line levels to the colorbar
        cbar.add_lines(cs)

        #plt.show()
        plt.draw()
        plt.pause(0.05)

In [ ]:
fig = init_figure()
Znew = np.copy(Z)

for it in range(1,n_iter+1):
    
    ls.calculate_collection_area(Z,NX,NY)
    ls.A = ls.A*(dx*dy)
    
    for i in range(NX):
        iL = np.mod(i-1,NX) # normally i-1 but observe p.b.c.
        iR = np.mod(i+1,NX) # normally i+1 but observe p.b.c.

        for j in range(NY):
            jD = np.mod(j-1,NY) # normally j-1 but observe p.b.c.
            jU = np.mod(j+1,NY) # normally j+1 but observe p.b.c.
  
            if ls.drain[i,j]>0: #regular pointm, not a pool, not a drainage point 
                ### use Z[i,j] and the indices i,iL,iR,j,jD,jU to derive 4 terms
                s1 = ... ### enter gradient term 's1'
                s2 = ... ### enter gradient term 's2'
                s3 = ... ### enter gradient term 's3'
                s4 = ... ### enter gradient term 's4'
                gradient = ... ### compute gradient from s1-s4
                
                #use gradient, uplift rate, erosion threshold, and drainage array size stored in ls.A[i,j]
                Psi_z = ... ### see Eq. (15) of Perron (2008) paper

            elif ls.drainage[i,j]>0: #this cell is a drainage point (it drains a pool)
                
                if (Z[i,j]>=Z[iR,j]) and ls.pool[iR,j]!=ls.drainage[i,j]: 
                    gradient = (Z[i,j]-Z[iR,j])/dx #pool is on my left, I drain to the right, use this gradiant
                elif (Z[i,j]>=Z[iL,j]) and ls.pool[iL,j]!=ls.drainage[i,j]:
                    gradient = (Z[i,j]-Z[iL,j])/dx
                elif (Z[i,j]>=Z[i,jU]) and ls.pool[i,jU]!=ls.drainage[i,j]:
                    gradient = (Z[i,j]-Z[i,jU])/dy
                elif (Z[i,j]>=Z[i,jD]) and ls.pool[i,jD]!=ls.drainage[i,j]:
                    gradient = (Z[i,j]-Z[i,jD])/dy
                else:
                    gradient = 0.02 # ??? This does happen (maybe when two pools merge)

                Psi_z = ... ### repeat Eq. (15) from above
                
            else: #this cell is a pool, assume it has some mass diffusion but no erosion!
                Psi_z = 0.
                
            if (Psi_z<0):
                Psi_z = 0. 

            # diffusion term, derive in terms of D,Z[:,:],dx,dy
            Phi_z = ...
           
            # insert Phi_z, Psi_z, and uplift terms to obtain new elevation Znew[i,j]
            Znew[i,j] = Z[i,j] + ( ... )*dt  

            dZdt= (Znew[i,j] - Z[i,j]) / dt
            CFL = abs(dZdt) * dt / min(dx,dy)
            if (CFL>1.0):
                print('\nWarning: Time step of',dt,'is probably too large. Safer would be:',dt/CFL)

            if (Znew[i,j]<0.):
                Znew[i,j] = 0. # yes, this does happen at the boundary when kept at zero
    
    Znew[0,:] = 0.0 # resets front boundary to 0
    Z = np.copy(Znew)
    
    ls.pool_check(Z,NX,NY)

    if (np.mod(it,10)==0): 
        print(it,end='')
        update_figure()
    else:
        print('.',end='')

update_figure()
print(' Simulation finished.')
